In [1]:
import torch
from src.nn.train import accuracy
from src.dataset.dataset import SparkifyDataset
from src.nn.Transformer import TransformerModel
import json
from src.preprocess.collate_fn import mat_collate_fn
from torch.utils.data import DataLoader

In [2]:
unique_pages_file = "data/unique_pages.json"
with open(unique_pages_file, "r") as f:
    unique_pages = json.load(f)

In [3]:
test_file = "data/test_dataset"

In [4]:
ds_test = SparkifyDataset(test_file)

In [5]:
model = TransformerModel(
    num_classes=len(unique_pages),
    nhead=8,
    num_layers=2
)

In [6]:
dl = DataLoader(ds_test, batch_size=8, collate_fn=mat_collate_fn)

In [21]:
sample = next(iter(dl))

In [22]:
emb, pos, targets, mask = sample

In [23]:
output = model(emb, pos, mask)

In [24]:
preds = output.argmax(dim=-1)

In [25]:
preds

tensor([[21, 21, 21,  ..., 21, 21, 21],
        [21, 21, 17,  ..., 21, 21, 21],
        [21, 21, 21,  ..., 21, 21, 21],
        ...,
        [21, 21, 21,  ..., 21, 21, 21],
        [21, 21, 21,  ..., 21, 21, 21],
        [21, 21, 21,  ..., 21, 21, 21]])

In [26]:
targets

tensor([[-1, -1, -1,  ..., -1, -1, -1],
        [-1, -1,  0,  ..., -1, -1, -1],
        [-1, -1, -1,  ..., -1, -1, -1],
        ...,
        [-1, -1, -1,  ..., -1, -1, -1],
        [-1, -1, -1,  ..., -1, -1, -1],
        [-1, -1, -1,  ..., -1, -1, -1]])

In [27]:
accuracy(preds, targets)

0.0